### Imports

In [1]:
import sqlite3
import pyterrier as pt
import os
from pathlib import Path
from bs4 import BeautifulSoup  # To clean the html
import warnings
warnings.filterwarnings("ignore")

if not pt.started():
    pt.init()

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
C:\Users\ajsko\AppData\Local\Temp\ipykernel_35260\2994591996.py:10: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


### SQLite DB for the Scraped Data

In [ ]:
conn = sqlite3.connect("../db/db")  # Path to database

c = conn.cursor()

In [ ]:
def extract_text(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    return soup.get_text()

In [ ]:
c.execute("SELECT id, html_content FROM webpages")
documents = []
for doc_id, html in c.fetchall():
    documents.append({"docno": doc_id, "text": extract_text(html)})

In [ ]:
documents[0]

### Indexing

In [ ]:
index_path = Path("../webscraper/index")
if os.path.exists(index_path):
    index = pt.IndexFactory.of(index_path)
else:
    indexer = pt.IterDictIndexer("../webscraper/index", meta={"docno": 20, "filename": 1024, "title": 1024}, meta_tags={"title":"title"})
    index_ref = indexer.index(documents)
    index = pt.IndexFactory.of(index_ref)

In [ ]:
print(index.getCollectionStatistics().toString())

In [ ]:
print("Available keys in MetaIndex:", index.getMetaIndex().getKeys())

In [ ]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
bm25.transform("LLM")